# Exercise on synaptic connectivity

In this worksheet, we explore ways to query synaptic connectivity from the MICrONS dataset and combine it with cell type annotations and other parameters of cells.

This tutorial assumes you have followed the [tutorial for programmatic access](https://github.com/sdorkenw/MICrONS_workshop/blob/main/tutorials/ProgrammaticAccess.ipynb), at least for setting up your CAVE account. 

## Setup and imports

In [1]:
# !pip install caveclient

In [2]:
import caveclient

import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

In [3]:
datastack_name = "minnie65_public"
client = caveclient.CAVEclient(datastack_name)

# One can pass the token directly to the client:
# my_token = "your token goes here"
# client = caveclient.CAVEclient(datastack_name, auth_token=my_token)

## 1. Sorted wiring diagram

In the tutorial, we queried the neuronal connectivity but the resulting wiring diagram showed little structure. Here, we want to combine the synaptic connecitivty with cell type classifications (from the table `aibs_metamodel_celltypes_v661`) to sort the wiring diagram. A suggested order of operations is

1. Get the proofreading table (`proofreading_status_public_release`)
2. Get the cell type table (`aibs_metamodel_celltypes_v661`)
3. Join/merge the proofreading table and the cell type table using `pd.merge` on `pt_root_id`
4. Lookup the synapses of the entries in the resulting table and create synapse matrix
5. Sort the rows and columns of the synapse matrix using the cell type information (see `sort_matrix_by_label()` below)
6. Plot using `sns.heatmap`
7. Bonus: Plot using `sns.clustermap` without clustering (`row_cluster=False`, `col_cluster=False`) and passing colors to `row_colors` and `col_colors` to indicate the cell types)

Tip: The logic for all steps is contained in the tutorial.

In [4]:
def sort_matrix_by_types(mat: pd.DataFrame, 
                         labels: pd.DataFrame, 
                         label_type_col: str = "cell_type", 
                         label_id_col: str = "pt_root_id"):
    """Sorts (synapse) matrix by labels.

    This function assumes a square synapse matrix!

    Args:
        mat: synapse matrix as pandas DataFrame
        labels: DataFrame with labels, e.g. the output of client.materialize.query_table('aibs_metamodel_celltypes_v661')
        label_type_col: column name in labels for cell types
        label_id_col: column name in labels for root ids

    Returns:
        mat_sorted: sorted matrix
        mat_labels: sorted labels; has the same length as matrix
    """
    assert mat.shape[0] == mat.shape[1]
    
    mat_sorted = mat.copy()
    
    mat_labels = np.array(labels.set_index(label_id_col).loc[mat.index][label_type_col])
    sorting = np.argsort(mat_labels)
    
    mat_sorted = mat_sorted.iloc[sorting].T.iloc[sorting].T

    return mat_sorted, mat_labels[sorting]

## 2. Distribution of synaptically targeted cell bodies

So far, we ignored that most neurons in the MICrONS dataset have well reconstructed dendrites (but not necessarily good axons). Here, we will use this feature of the dataset by looking at all the targets of a single neuron and find the soma locations of their postsynaptic targets.

First select a neuron to analyze - consider visualizing it in Neuroglancer and DashApss first. We suggest a neuron below but feel free to change that

In [12]:
root_id = 864691135155894884

The goal is to create a scatter plot that shows the cell body location of all the neurons targeted by our neuron of choice. We will use the synapse and the cell type tables to obtain the necessary information. Join them together to add the information about the postsynaptic cell body location to all synapses.

Bonus: Get creative with the annotations in the scatter plot. For instance, use cell types to color the postsynaptic target dots.

### Bonus exercise: Synapse sorting

Synapse sorting describes the effect where different postsynaptic neurons are targeted more proximal or distal to the soma. For instance, multiple studies (e.g. [Schmid et al., 2017](https://www.nature.com/articles/nature24005) Fig. 2, [Dorkenwald, Li et al., 2023](https://www.nature.com/articles/s41592-023-02059-8) Fig. 6k-l) found that pyramidal cells target predominantly inhibitory neurons along their proximal axon and excitatory neurons along their distal axon. These studies used the path length along the soma, but some of the effect can be observed using eucliedian distance between the presynaptic soma and the synapse. 

Create a histogram of the distances of synapses onto inhibitory targets and one of the distances synapses onto excitatory targets and compare.

## 3. Functional connectomics

Lastly, we want to relate synaptic connectivity to structural measurements of the cells. This is an open area of analysis but the most common type of analysis is to quantify "like-to-like" connectivity. "Like-to-like" refers to an effect where neurons with similar properties tend to be more connected. For instance [Ko et al., 2011](https://www.nature.com/articles/nature09880) find that "We found that connection probability was related to the similarity of visually driven neuronal activity." 


Using difference in preferred direction as a measurement of functional similarity, evaluate whether neurons in the MICrONS dataset exhibit like-to-like connectivity. To make things simpler, consider a null model that uses a randomized version of the synapse table. The effect is likely dependent on cell types; consider only analyzing the connections between `23P` cells first.